! alle Pakete installieren

# Pakete importieren 

In [ ]:
import random
import statistics

# Code

## 1) Parameter des Systems

In [ ]:
LAMBDA_ANKUNFT_PRO_WOCHE = 20   # Ø neue Anmeldungen pro Woche
SIMULATIONS_DAUER_WOCHEN = 200  # über wie viele Wochen simuliert wird

# Kapazität
ANZAHL_THERAPEUTEN = 5          # Anzahl Therapeut*innen

# Therapiedauer (Servicezeit) in Wochen
MIN_THERAPIEDAUER = 10          # z. B. 10 Sitzungen à 1 Woche
MAX_THERAPIEDAUER = 40          # z. B. 40 Sitzungen

# Zufalls-Seed für Reproduzierbarkeit (ändern(?))
random.seed(42)

## 2) Funktionen

In [ ]:
def generiere_ankunftszeiten(lambda_pro_woche, sim_dauer_wochen):
    """
    Erzeugt eine Liste aufsteigender Ankunftszeiten (in Wochen)
    auf Basis eines Poisson-Prozesses (exponentielle Zwischenankunftszeiten).
    """
    ankunftszeiten = []
    aktuelle_zeit = 0.0

    # mittlere Zwischenankunftszeit
    mean_interarrival = 1.0 / lambda_pro_woche  # Wochen

    while aktuelle_zeit < sim_dauer_wochen:
        # Exponentiell verteilte Zwischenankunftszeit
        delta = random.expovariate(1.0 / mean_interarrival)
        aktuelle_zeit += delta
        if aktuelle_zeit > sim_dauer_wochen:
            break
        ankunftszeiten.append(aktuelle_zeit)

    return ankunftszeiten


def ziehe_therapiedauer(min_wochen, max_wochen):
    """
    Ziehe eine zufällige Therapiedauer (in Wochen).
    Hier: einfache Gleichverteilung zwischen min und max.
    Später kannst du z.B. Normal- oder andere Verteilungen verwenden.
    """
    return random.uniform(min_wochen, max_wochen)


## 3) Zielgrößen

## 4) Simulationslauf

In [ ]:
def simuliere_warteschlange():
    # 3.1 Ankunftszeiten generieren
    ankunftszeiten = generiere_ankunftszeiten(
        LAMBDA_ANKUNFT_PRO_WOCHE,
        SIMULATIONS_DAUER_WOCHEN
    )

    # nächster Zeitpunkt, an dem jede/r Therapeut*in frei ist
    # Index: Therapeut, Wert: Zeit (in Wochen)
    naechst_frei = [0.0 for _ in range(ANZAHL_THERAPEUTEN)]

    # Listen zur Auswertung
    wartezeiten = []     # nur echte Wartezeit (bis Therapiebeginn)
    systemzeiten = []    # Zeit von Anmeldung bis Therapieende

    # zur Auslastungsberechnung: gesamte Behandlungszeit je Therapeut
    gesamt_behandlungszeit_pro_therapeut = [0.0 for _ in range(ANZAHL_THERAPEUTEN)]

    for ankunft in ankunftszeiten:
        # Therapiedauer für diesen Patienten
        servicezeit = ziehe_therapiedauer(
            MIN_THERAPIEDAUER, MAX_THERAPIEDAUER
        )

        # 3.2 Finde den Therapeuten, der am frühesten frei ist
        therapeut_index = min(range(ANZAHL_THERAPEUTEN),
                              key=lambda i: naechst_frei[i])
        frueheste_frei_zeit = naechst_frei[therapeut_index]

        # 3.3 Therapiebeginn: max(Ankunft, frühester Frei-Zeitpunkt)
        therapiebeginn = max(ankunft, frueheste_frei_zeit)
        therapieende = therapiebeginn + servicezeit

        # 3.4 Warte- und Systemzeit berechnen
        wartezeit = therapiebeginn - ankunft
        systemzeit = therapieende - ankunft

        wartezeiten.append(wartezeit)
        systemzeiten.append(systemzeit)

        # 3.5 Kalender des Therapeuten aktualisieren
        naechst_frei[therapeut_index] = therapieende
        gesamt_behandlungszeit_pro_therapeut[therapeut_index] += servicezeit

    # ============================================
    # 4. Kennzahlen berechnen
    # ============================================

    # Durchschnittliche Wartezeit & Systemzeit
    mittlere_wartezeit = statistics.mean(wartezeiten) if wartezeiten else 0.0
    mittlere_systemzeit = statistics.mean(systemzeiten) if systemzeiten else 0.0

    # Median & ein paar Quantile (optional)
    wartezeiten_sortiert = sorted(wartezeiten)
    def quantil(values, q):
        if not values:
            return 0.0
        idx = int(q * (len(values) - 1))
        return values[idx]

    median_warte = quantil(wartezeiten_sortiert, 0.5)
    q75_warte = quantil(wartezeiten_sortiert, 0.75)
    q90_warte = quantil(wartezeiten_sortiert, 0.90)

    # Auslastung der Therapeut*innen
    # Annahme: Simulation läuft SIMULATIONS_DAUER_WOCHEN,
    # und jede/r Therapeut*in könnte theoretisch durchgehend arbeiten.
    auslastungen = [
        behandlungszeit / SIMULATIONS_DAUER_WOCHEN
        for behandlungszeit in gesamt_behandlungszeit_pro_therapeut
    ]
    mittlere_auslastung = statistics.mean(auslastungen) if auslastungen else 0.0

    # Anteil mit Wartezeit > X Wochen (z. B. 12 und 24 Wochen)
    def anteil_mit_wartezeit_groesser_x(x):
        if not wartezeiten:
            return 0.0
        return sum(w > x for w in wartezeiten) / len(wartezeiten)

    anteil_ueber_12 = anteil_mit_wartezeit_groesser_x(12)
    anteil_ueber_24 = anteil_mit_wartezeit_groesser_x(24)

## 5) Kennzahlen berechnen 

In [ ]:
# Durchschnittliche Wartezeit & Systemzeit
    mittlere_wartezeit = statistics.mean(wartezeiten) if wartezeiten else 0.0
    mittlere_systemzeit = statistics.mean(systemzeiten) if systemzeiten else 0.0

    # Median & ein paar Quantile (optional)
    wartezeiten_sortiert = sorted(wartezeiten)
    def quantil(values, q):
        if not values:
            return 0.0
        idx = int(q * (len(values) - 1))
        return values[idx]

    median_warte = quantil(wartezeiten_sortiert, 0.5)
    q75_warte = quantil(wartezeiten_sortiert, 0.75)
    q90_warte = quantil(wartezeiten_sortiert, 0.90)

    # Auslastung der Therapeut*innen
    # Annahme: Simulation läuft SIMULATIONS_DAUER_WOCHEN,
    # und jede/r Therapeut*in könnte theoretisch durchgehend arbeiten.
    auslastungen = [
        behandlungszeit / SIMULATIONS_DAUER_WOCHEN
        for behandlungszeit in gesamt_behandlungszeit_pro_therapeut
    ]
    mittlere_auslastung = statistics.mean(auslastungen) if auslastungen else 0.0

    # Anteil mit Wartezeit > X Wochen (z. B. 12 und 24 Wochen)
    def anteil_mit_wartezeit_groesser_x(x):
        if not wartezeiten:
            return 0.0
        return sum(w > x for w in wartezeiten) / len(wartezeiten)

    anteil_ueber_12 = anteil_mit_wartezeit_groesser_x(12)
    anteil_ueber_24 = anteil_mit_wartezeit_groesser_x(24)

## 6) Ergbenisse ausgeben

In [ ]:
print("=== Ergebnisse der Simulation (Therapie-Warteschlange) ===")
    print(f"Anzahl Therapeut*innen:            {ANZAHL_THERAPEUTEN}")
    print(f"Simulierte Zeit:                   {SIMULATIONS_DAUER_WOCHEN:.1f} Wochen")
    print(f"Anzahl Anmeldungen:                {len(ankunftszeiten)}")

    print("\nWartezeiten (in Wochen)")
    print(f"  Mittelwert:                      {mittlere_wartezeit:.2f}")
    print(f"  Median:                          {median_warte:.2f}")
    print(f"  75%-Quantil:                     {q75_warte:.2f}")
    print(f"  90%-Quantil:                     {q90_warte:.2f}")
    print(f"  Anteil > 12 Wochen:              {anteil_ueber_12*100:.1f} %")
    print(f"  Anteil > 24 Wochen:              {anteil_ueber_24*100:.1f} %")

    print("\nGesamtzeit im System (Anmeldung bis Therapieende)")
    print(f"  Mittelwert:                      {mittlere_systemzeit:.2f} Wochen")

    print("\nAuslastung der Therapeut*innen")
    for i, rho in enumerate(auslastungen):
        print(f"  Therapeut {i+1}:                 {rho*100:.1f} %")
    print(f"  Durchschnittliche Auslastung:    {mittlere_auslastung*100:.1f} %")
    
    if __name__ == "__main__":
    simuliere_warteschlange()

# Verbesserungen

=== Modell: Therapie (1 Sitzung/Woche, 24 Wochen), Kapazität = Sitzungen/Woche ===
Therapeut*innen:                14,600
Sitzungen/Woche pro Therapeut:  35
Kapazität (aktive Patient*innen): 511,000
Ankunftsrate:                   20,000 / Woche
Simulierte Ankünfte:            5,000,000
Warm-up ignoriert:              50,000
Reservoir (Quantile):           200,000

--- Run seed=42 (N=4,950,000) ---
Mean wait (weeks):      0.000  (SE=0.0000, CI95=[0.000, 0.000])
Quantiles wait:         Q75=0.000, Q90=0.000, Q95=0.000
Share wait > 12 weeks:  0.00%
Share wait > 24 weeks:  0.00%
Mean system time:       24.000 weeks (SE=0.0000)

--- Run seed=43 (N=4,950,000) ---
Mean wait (weeks):      0.000  (SE=0.0000, CI95=[0.000, 0.000])
Quantiles wait:         Q75=0.000, Q90=0.000, Q95=0.000
Share wait > 12 weeks:  0.00%
Share wait > 24 weeks:  0.00%
Mean system time:       24.000 weeks (SE=0.0000)

--- Run seed=44 (N=4,950,000) ---
Mean wait (weeks):      0.000  (SE=0.0000, CI95=[0.000, 0.000])
Quanti

## a)

In [2]:
# pip install simpy numpy pandas

import simpy
import numpy as np
import pandas as pd

# -----------------------------
# Parameter (aus Screenshot + sinnvolle Defaults)
# -----------------------------
YEAR_WEEKS = 52
SEED = 42
rng = np.random.default_rng(SEED)

TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557  # "Ja: 84.557 (88,82%)"
P_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR

N_THERAPISTS = 9000

THERAPY_SESSIONS = 24
SESSION_FREQUENCY_PER_WEEK = 1  # 1 Sitzung pro Woche
THERAPY_DURATION_WEEKS = THERAPY_SESSIONS / SESSION_FREQUENCY_PER_WEEK  # 24 Wochen

# Ankunftsprozess: Rate pro Woche wächst exponentiell: lambda(t) = lambda0 * exp(growth * t)
# - lambda0 wird so gewählt, dass im Mittel TOTAL_PATIENTS_YEAR im Jahr ankommen.
# - growth setzt die Stärke des Wachstums (0 = konstant)
ARRIVAL_GROWTH_PER_WEEK = 0.02  # <-- anpassen: 2% Wachstum pro Woche ist stark, nur Default!

# -----------------------------
# Hilfsfunktionen
# -----------------------------
def integral_exp_rate(lambda0, g, T):
    """Integral_0^T lambda0*exp(g t) dt"""
    if abs(g) < 1e-12:
        return lambda0 * T
    return lambda0 * (np.exp(g * T) - 1.0) / g

def solve_lambda0_for_total(total_expected, g, T):
    """Wähle lambda0 so, dass E[Ankünfte in (0,T)] = total_expected."""
    if abs(g) < 1e-12:
        return total_expected / T
    return total_expected * g / (np.exp(g * T) - 1.0)

LAMBDA0_PER_WEEK = solve_lambda0_for_total(TOTAL_PATIENTS_YEAR, ARRIVAL_GROWTH_PER_WEEK, YEAR_WEEKS)

def current_arrival_rate(env_now_week):
    return LAMBDA0_PER_WEEK * np.exp(ARRIVAL_GROWTH_PER_WEEK * env_now_week)

# Thinning für nicht-homogenen Poisson-Prozess
def nhpp_arrivals(env, out_store, T_end, rng):
    """
    Erzeuge Ankunftszeiten bis T_end mit Thinning.
    out_store bekommt Events (arrival_time).
    """
    t = 0.0
    # obere Schranke der Rate im Zeitintervall: bei exponentiellem Wachstum max am Ende
    lambda_max = current_arrival_rate(T_end)

    while True:
        # Kandidaten-Interarrival für homog. Poisson mit Rate lambda_max
        dt = rng.exponential(scale=1.0 / lambda_max)
        t += dt
        if t > T_end:
            break

        # Akzeptanzwahrscheinlichkeit
        lam_t = current_arrival_rate(t)
        if rng.random() < (lam_t / lambda_max):
            yield out_store.put(t)

def patient(env, pid, therapists, results, rng):
    """
    Patient kommt an, entscheidet ggf. ob er/sie wirklich startet, wartet auf Therapeut*in,
    belegt Therapeut*in für THERAPY_DURATION_WEEKS.
    """
    arrival = env.now

    # "Ja/Nein"-Gate (aus Screenshot)
    if rng.random() > P_START_THERAPY:
        results["dropped"].append(1)
        return
    results["dropped"].append(0)

    with therapists.request() as req:
        yield req
        start = env.now
        wait = start - arrival

        # Therapie dauert 24 Wochen (Default)
        yield env.timeout(THERAPY_DURATION_WEEKS)
        end = env.now

    results["wait_weeks"].append(wait)
    results["sojourn_weeks"].append(end - arrival)  # Warten + Therapie
    results["start_week"].append(start)
    results["arrival_week"].append(arrival)

def system(env, therapists, arrival_times_store, T_end, results, rng, queue_trace_every=1.0):
    """
    Startet Patient*innen bei ihren Ankunftszeiten und zeichnet Queue/Utilization grob auf.
    """
    pid = 0

    # Monitoring-Prozess
    def monitor():
        while env.now < T_end:
            # queue length = Anzahl wartender Requests
            q_len = len(therapists.queue)
            in_service = therapists.count
            results["queue_len_trace"].append((env.now, q_len))
            results["in_service_trace"].append((env.now, in_service))
            yield env.timeout(queue_trace_every)

    env.process(monitor())

    while True:
        t = yield arrival_times_store.get()
        if t > T_end:
            break
        # springe zur Ankunftszeit (falls wir "vorne" sind, timeout)
        if env.now < t:
            yield env.timeout(t - env.now)

        pid += 1
        env.process(patient(env, pid, therapists, results, rng))

# -----------------------------
# Run
# -----------------------------
def run_simulation():
    env = simpy.Environment()
    therapists = simpy.Resource(env, capacity=N_THERAPISTS)

    arrival_times_store = simpy.Store(env)
    results = {
        "wait_weeks": [],
        "sojourn_weeks": [],
        "arrival_week": [],
        "start_week": [],
        "dropped": [],
        "queue_len_trace": [],
        "in_service_trace": [],
    }

    env.process(nhpp_arrivals(env, arrival_times_store, YEAR_WEEKS, rng))
    env.process(system(env, therapists, arrival_times_store, YEAR_WEEKS, results, rng))

    env.run(until=YEAR_WEEKS + THERAPY_DURATION_WEEKS + 1)  # damit begonnene Therapien zu Ende laufen können

    # -----------------------------
    # Auswertung
    # -----------------------------
    df = pd.DataFrame({
        "arrival_week": results["arrival_week"],
        "start_week": results["start_week"],
        "wait_weeks": results["wait_weeks"],
        "sojourn_weeks": results["sojourn_weeks"],
    })

    dropped_rate = np.mean(results["dropped"])  # 1 = dropped
    started = len(df)

    # Monitoring
    q_trace = pd.DataFrame(results["queue_len_trace"], columns=["week", "queue_len"])
    s_trace = pd.DataFrame(results["in_service_trace"], columns=["week", "in_service"])
    utilization_est = (s_trace["in_service"].mean() / N_THERAPISTS) if len(s_trace) else np.nan

    summary = {
        "expected_arrivals_year": TOTAL_PATIENTS_YEAR,
        "lambda0_per_week": LAMBDA0_PER_WEEK,
        "growth_per_week": ARRIVAL_GROWTH_PER_WEEK,
        "therapists": N_THERAPISTS,
        "therapy_duration_weeks": THERAPY_DURATION_WEEKS,
        "started_patients": started,
        "dropped_share": dropped_rate,
        "mean_wait_weeks": float(df["wait_weeks"].mean()) if started else np.nan,
        "p90_wait_weeks": float(df["wait_weeks"].quantile(0.90)) if started else np.nan,
        "p95_wait_weeks": float(df["wait_weeks"].quantile(0.95)) if started else np.nan,
        "max_wait_weeks": float(df["wait_weeks"].max()) if started else np.nan,
        "mean_queue_len": float(q_trace["queue_len"].mean()) if len(q_trace) else np.nan,
        "max_queue_len": int(q_trace["queue_len"].max()) if len(q_trace) else None,
        "utilization_mean": float(utilization_est),
    }

    return summary, df, q_trace, s_trace

if __name__ == "__main__":
    summary, df, q_trace, s_trace = run_simulation()
    print("=== SUMMARY ===")
    for k, v in summary.items():
        print(f"{k}: {v}")

    print("\nBeispiel (erste 10 gestartete Patient*innen):")
    print(df.head(10))

=== SUMMARY ===
expected_arrivals_year: 95201
lambda0_per_week: 1040.893445152519
growth_per_week: 0.02
therapists: 9000
therapy_duration_weeks: 24.0
started_patients: 22861
dropped_share: 0.11090457194176988
mean_wait_weeks: 13.0896265898785
p90_wait_weeks: 32.28153640143486
p95_wait_weeks: 32.61657931976839
max_wait_weeks: 32.89300809477673
mean_queue_len: 22138.076923076922
max_queue_len: 61604
utilization_mean: 0.9023119658119658

Beispiel (erste 10 gestartete Patient*innen):
   arrival_week  start_week  wait_weeks  sojourn_weeks
0      0.003191    0.003191         0.0           24.0
1      0.003222    0.003222         0.0           24.0
2      0.004799    0.004799         0.0           24.0
3      0.004953    0.004953         0.0           24.0
4      0.005543    0.005543         0.0           24.0
5      0.006779    0.006779         0.0           24.0
6      0.007545    0.007545         0.0           24.0
7      0.007654    0.007654         0.0           24.0
8      0.007774    0

In [3]:
import random
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional

# -----------------------------
# VT Kapazität (deine Daten)
# -----------------------------
THERAPISTS_TOTAL = 9000

THERAPISTS_PART = 5490   # 61%
THERAPISTS_FULL = 3510   # 39%

SESSIONS_PER_WEEK_PART = 9    # ~20 Std
SESSIONS_PER_WEEK_FULL = 19   # ~40 Std

SLOTS_PER_WEEK = THERAPISTS_PART * SESSIONS_PER_WEEK_PART + THERAPISTS_FULL * SESSIONS_PER_WEEK_FULL
# = 116_190

# Therapie-Logik
THERAPY_SESSIONS = 24
SESSIONS_PER_PATIENT_PER_WEEK = 1
THERAPY_DURATION_WEEKS = THERAPY_SESSIONS // SESSIONS_PER_PATIENT_PER_WEEK  # 24 Wochen

# -----------------------------
# Arrival Modell
# -----------------------------
@dataclass
class ArrivalConfig:
    mode: str = "poisson"  # "poisson" oder "uniform"

    # Poisson: arrivals ~ Poisson(mean)
    poisson_mean: float = 1000.0

    # Uniform: arrivals ~ randint(low, high)
    uniform_low: int = 800
    uniform_high: int = 1200

# -----------------------------
# Simulation
# -----------------------------
@dataclass
class SimConfig:
    weeks: int = 52
    seed: int = 42
    slots_per_week: int = SLOTS_PER_WEEK
    therapy_duration_weeks: int = THERAPY_DURATION_WEEKS
    arrivals: ArrivalConfig = field(default_factory=ArrivalConfig)

@dataclass
class SimResults:
    started: int
    arrived: int
    still_waiting_end: int
    mean_wait_weeks: float
    p90_wait_weeks: float
    max_wait_weeks: int
    mean_queue_len: float
    max_queue_len: int
    mean_utilization: float
    trace_queue: List[Tuple[int, int]]
    trace_util: List[Tuple[int, float]]

def poisson(lmbda: float) -> int:
    """Simple Poisson sampler (Knuth). OK für moderate lmbda; für sehr große lmbda besser numpy."""
    import math
    L = math.exp(-lmbda)
    k = 0
    p = 1.0
    while p > L:
        k += 1
        p *= random.random()
    return k - 1

def sample_arrivals(week: int, cfg: ArrivalConfig) -> int:
    if cfg.mode == "poisson":
        return max(0, poisson(cfg.poisson_mean))
    elif cfg.mode == "uniform":
        return random.randint(cfg.uniform_low, cfg.uniform_high)
    else:
        raise ValueError("arrivals.mode muss 'poisson' oder 'uniform' sein")

def run_sim(cfg: SimConfig) -> SimResults:
    random.seed(cfg.seed)

    # Queue speichert Ankunftswochen (für Wartedauer)
    queue: List[int] = []

    # Active therapies: Liste verbleibender Wochen (jede belegt 1 Slot/Woche)
    active: List[int] = []

    wait_times: List[int] = []
    trace_queue: List[Tuple[int, int]] = []
    trace_util: List[Tuple[int, float]] = []

    total_arrived = 0
    total_started = 0
    queue_len_sum = 0
    util_sum = 0.0
    util_count = 0

    # Simulieren über "weeks + therapy_duration" damit wir Auslastung/Abbau sehen können
    total_weeks_sim = cfg.weeks + cfg.therapy_duration_weeks

    for w in range(total_weeks_sim):
        # 1) Therapien eine Woche "ablaufen" lassen
        if active:
            active = [rem - 1 for rem in active]
            active = [rem for rem in active if rem > 0]

        # 2) Neue Arrivals nur im Beobachtungsjahr (erste cfg.weeks Wochen)
        if w < cfg.weeks:
            a = sample_arrivals(w, cfg.arrivals)
            total_arrived += a
            # alle kommen in die Warteschlange
            queue.extend([w] * a)

        # 3) Kapazität: Slots pro Woche minus belegte Slots durch aktive Therapien
        occupied_slots = len(active)  # 1 Slot pro aktiver Patient*in pro Woche
        free_slots = max(0, cfg.slots_per_week - occupied_slots)

        # 4) Start: so viele wie freie Slots und Queue hergibt
        can_start = min(free_slots, len(queue))
        for _ in range(can_start):
            arrival_week = queue.pop(0)
            wait = w - arrival_week
            wait_times.append(wait)
            active.append(cfg.therapy_duration_weeks)
            total_started += 1

        # 5) Monitoring
        qlen = len(queue)
        trace_queue.append((w, qlen))
        queue_len_sum += qlen

        util = (occupied_slots / cfg.slots_per_week) if cfg.slots_per_week > 0 else 0.0
        trace_util.append((w, util))
        util_sum += util
        util_count += 1

    # Kennzahlen
    if wait_times:
        wt_sorted = sorted(wait_times)
        mean_wait = sum(wait_times) / len(wait_times)
        p90 = wt_sorted[int(0.9 * (len(wt_sorted) - 1))]
        max_wait = wt_sorted[-1]
    else:
        mean_wait, p90, max_wait = 0.0, 0.0, 0

    mean_queue_len = queue_len_sum / total_weeks_sim
    max_queue_len = max(q for _, q in trace_queue) if trace_queue else 0
    mean_util = util_sum / util_count if util_count else 0.0

    return SimResults(
        started=total_started,
        arrived=total_arrived,
        still_waiting_end=len(queue),
        mean_wait_weeks=mean_wait,
        p90_wait_weeks=float(p90),
        max_wait_weeks=int(max_wait),
        mean_queue_len=float(mean_queue_len),
        max_queue_len=int(max_queue_len),
        mean_utilization=float(mean_util),
        trace_queue=trace_queue,
        trace_util=trace_util,
    )

if __name__ == "__main__":
    cfg = SimConfig(
        weeks=52,
        seed=42,
        slots_per_week=SLOTS_PER_WEEK,
        therapy_duration_weeks=THERAPY_DURATION_WEEKS,
        arrivals=ArrivalConfig(
            mode="poisson",      # "poisson" oder "uniform"
            poisson_mean=2000.0, # <-- HIER einstellen (Durchschnitt Arrivals/Woche)
            uniform_low=1500,
            uniform_high=2500
        ),
    )

    res = run_sim(cfg)

    print("=== VT Simulation ===")
    print(f"SLOTS_PER_WEEK: {cfg.slots_per_week:,}")
    print(f"THERAPY_DURATION_WEEKS: {cfg.therapy_duration_weeks}")
    print(f"ARRIVALS_MODE: {cfg.arrivals.mode}")
    if cfg.arrivals.mode == "poisson":
        print(f"POISSON_MEAN: {cfg.arrivals.poisson_mean}")
    else:
        print(f"UNIFORM_RANGE: [{cfg.arrivals.uniform_low}, {cfg.arrivals.uniform_high}]")

    print("\n--- Ergebnis (Jahr + Abbauphase) ---")
    print(f"Arrived (Jahr): {res.arrived:,}")
    print(f"Started: {res.started:,}")
    print(f"Still waiting at end: {res.still_waiting_end:,}")
    print(f"Mean wait (weeks): {res.mean_wait_weeks:.2f}")
    print(f"P90 wait (weeks): {res.p90_wait_weeks:.0f}")
    print(f"Max wait (weeks): {res.max_wait_weeks}")
    print(f"Mean queue length: {res.mean_queue_len:,.0f}")
    print(f"Max queue length: {res.max_queue_len:,}")
    print(f"Mean utilization: {res.mean_utilization*100:.1f}%")

=== VT Simulation ===
SLOTS_PER_WEEK: 116,100
THERAPY_DURATION_WEEKS: 24
ARRIVALS_MODE: poisson
POISSON_MEAN: 2000.0

--- Ergebnis (Jahr + Abbauphase) ---
Arrived (Jahr): 38,872
Started: 38,872
Still waiting at end: 0
Mean wait (weeks): 0.00
P90 wait (weeks): 0
Max wait (weeks): 0
Mean queue length: 0
Max queue length: 0
Mean utilization: 10.1%


In [4]:
# pip install simpy numpy

import simpy
import numpy as np

# -------------------------
# VT Daten (deins)
# -------------------------
TEILZEIT = 5490
VOLLZEIT = 3510

SLOTS_TEILZEIT = 9   # Sitzungen pro Woche
SLOTS_VOLLZEIT = 19  # Sitzungen pro Woche

SLOTS_PRO_WOCHE = TEILZEIT * SLOTS_TEILZEIT + VOLLZEIT * SLOTS_VOLLZEIT  # 116_190

THERAPIE_DAUER_WOCHEN = 24   # 24 Sitzungen, 1/Woche

# -------------------------
# Simulation-Parameter
# -------------------------
WOCHEN = 52
SEED = 42

# "Arrival per week: zufällig"
# -> arrivals ~ Poisson(mean_arrivals_per_week)
MEAN_ARRIVALS_PER_WEEK = 2000  # <-- HIER anpassen

# Ergebnisse sammeln (wie im PDF: wait_times Liste)  [oai_citation:2‡KAP_4_Dynamische_Simulation.pdf](sediment://file_0000000022f8720eb4886c6329c43dba)
wait_times = []

def patient_process(env, name, slots, rng):
    """Entity-Logik wie im PDF: ankommen -> Ressource anfordern -> warten -> Service -> fertig.  [oai_citation:3‡KAP_4_Dynamische_Simulation.pdf](sediment://file_0000000022f8720eb4886c6329c43dba)"""
    t_ankunft = env.now

    # Ressource anfordern (Slot für diese Woche)
    with slots.request() as req:
        yield req  # warten bis Slot frei ist
        t_start = env.now
        wait_times.append(t_start - t_ankunft)

        # Therapie "blockiert" 1 Slot pro Woche für 24 Wochen
        # (vereinfacht als durchgehende Belegung über 24 Wochen)
        yield env.timeout(THERAPIE_DAUER_WOCHEN)

def setup(env, slots, rng):
    """Erzeugt neue Patient*innen (Arrivals). Wie im PDF 'setup' generiert Entities.  [oai_citation:4‡KAP_4_Dynamische_Simulation.pdf](sediment://file_0000000022f8720eb4886c6329c43dba)"""
    i = 0

    # Wir modellieren "pro Woche kommt eine zufällige Anzahl"
    for w in range(WOCHEN):
        # springe zur nächsten Woche
        yield env.timeout(1)

        # zufällige Arrivals in dieser Woche
        arrivals = rng.poisson(MEAN_ARRIVALS_PER_WEEK)

        for _ in range(arrivals):
            i += 1
            env.process(patient_process(env, f"Patient {i}", slots, rng))

if __name__ == "__main__":
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    # Resource = Sitzungs-Slots pro Woche (Kapazität)
    slots = simpy.Resource(env, capacity=SLOTS_PRO_WOCHE)

    env.process(setup(env, slots, rng))

    # laufen lassen: 52 Wochen + genug Zeit, damit gestartete Therapien fertig werden
    env.run(until=WOCHEN + THERAPIE_DAUER_WOCHEN + 1)

    # Auswertung wie im PDF (mean, quantile, max)  [oai_citation:5‡KAP_4_Dynamische_Simulation.pdf](sediment://file_0000000022f8720eb4886c6329c43dba)
    if len(wait_times) == 0:
        print("Keine Patient*innen gestartet.")
    else:
        wt = np.array(wait_times)
        print("=== VT Ergebnis ===")
        print(f"SLOTS_PRO_WOCHE: {SLOTS_PRO_WOCHE:,}")
        print(f"MEAN_ARRIVALS_PER_WEEK: {MEAN_ARRIVALS_PER_WEEK}")
        print(f"Anzahl gestartete Patient*innen: {len(wt):,}")
        print(f"Ø Wartezeit (Wochen): {wt.mean():.2f}")
        print(f"95%-Quantil Wartezeit: {np.quantile(wt, 0.95):.2f}")
        print(f"Max Wartezeit: {wt.max():.2f}")

=== VT Ergebnis ===
SLOTS_PRO_WOCHE: 116,100
MEAN_ARRIVALS_PER_WEEK: 2000
Anzahl gestartete Patient*innen: 103,839
Ø Wartezeit (Wochen): 0.00
95%-Quantil Wartezeit: 0.00
Max Wartezeit: 0.00


In [5]:
# VT Simulation (alles-in-einem, copy & paste)
# pip install simpy numpy
#
# Modell (einfach, aber verbessert):
# - Kapazität = Sitzungs-Slots pro Woche (Teilzeit + Vollzeit)
# - Arrivals pro Woche = zufällig (Poisson) + optionales Wachstum
# - Therapiestart = randomisiert (Start-Delay)
# - Therapie = 24 einzelne Termine (1/Woche), jeder Termin belegt einen Slot kurz
# - Parallelität entsteht automatisch über viele Slots (Resource capacity)

import simpy
import numpy as np

# -------------------------
# 1) VT Daten
# -------------------------
TEILZEIT_THERAPEUTEN = 5490   # 61%
VOLLZEIT_THERAPEUTEN = 3510   # 39%

SLOTS_PRO_WOCHE_TEILZEIT = 9  # 20 Std -> 9 Sitzungen/Woche
SLOTS_PRO_WOCHE_VOLLZEIT = 19 # 40 Std -> 19 Sitzungen/Woche

SLOTS_PRO_WOCHE = (
    TEILZEIT_THERAPEUTEN * SLOTS_PRO_WOCHE_TEILZEIT
    + VOLLZEIT_THERAPEUTEN * SLOTS_PRO_WOCHE_VOLLZEIT
)  # = 116_190

# Therapie: 24 Sitzungen, 1 Sitzung / Woche
THERAPIE_SITZUNGEN = 24

# Dauer einer einzelnen Sitzung als Wochenanteil (nur damit es "Servicezeit" gibt)
# Du kannst das auch auf 0 setzen, dann ist die Sitzung quasi "instant".
SESSION_DURATION = 1 / 19  # ~0.0526 Woche

# -------------------------
# 2) Simulation Parameter
# -------------------------
WOCHEN_JAHR = 52
SEED = 42

# Arrivals pro Woche: zufällig (Poisson)
BASE_ARRIVALS_MEAN = 2000     # <-- HIER anpassen (Durchschnitt Woche 1)
GROWTH_PER_WEEK = 30          # <-- optional: mittleres Wachstum pro Woche (0 = kein Wachstum)

# Variation / Randomisierung Therapiestart:
# - "uniform": 0..MAX_START_DELAY_WEEKS Wochen
# - "exponential": exp(mean=START_DELAY_MEAN_WEEKS)
START_DELAY_MODE = "uniform"  # "uniform" oder "exponential"
MAX_START_DELAY_WEEKS = 4
START_DELAY_MEAN_WEEKS = 1.5

# Monitoring (für einfache Auswertung)
queue_len_trace = []
util_trace = []
wait_times = []

# -------------------------
# 3) Prozesse
# -------------------------
def sample_start_delay(rng: np.random.Generator) -> float:
    if START_DELAY_MODE == "uniform":
        return float(rng.integers(0, MAX_START_DELAY_WEEKS + 1))  # inkl. MAX
    if START_DELAY_MODE == "exponential":
        return float(rng.exponential(START_DELAY_MEAN_WEEKS))
    raise ValueError("START_DELAY_MODE muss 'uniform' oder 'exponential' sein")

def patient(env: simpy.Environment, slots: simpy.Resource, rng: np.random.Generator):
    t_arrival = env.now

    # Randomisierung beim Therapiestart
    delay = sample_start_delay(rng)
    yield env.timeout(delay)

    # Therapie als 24 einzelne Termine/Slots (1 pro Woche)
    for s in range(THERAPIE_SITZUNGEN):
        with slots.request() as req:
            yield req

            # Wartezeit nur bis zur ersten Sitzung messen
            if s == 0:
                wait_times.append(env.now - t_arrival)

            # "Sitzung dauert kurz" (Servicezeit)
            if SESSION_DURATION > 0:
                yield env.timeout(SESSION_DURATION)

        # Rest der Woche bis zum nächsten Termin
        rest = 1.0 - SESSION_DURATION
        if rest > 0:
            yield env.timeout(rest)

def arrivals(env: simpy.Environment, slots: simpy.Resource, rng: np.random.Generator):
    # pro Woche kommt eine zufällige Anzahl Patient*innen
    for w in range(WOCHEN_JAHR):
        yield env.timeout(1.0)

        mean = BASE_ARRIVALS_MEAN + GROWTH_PER_WEEK * w
        n = int(rng.poisson(max(0.0, mean)))

        for _ in range(n):
            env.process(patient(env, slots, rng))

def monitor(env: simpy.Environment, slots: simpy.Resource):
    # misst pro Woche Queue-Länge und Auslastung
    while True:
        q_len = len(slots.queue)
        in_service = slots.count  # aktuell belegte "Slots" (laufende Requests)
        util = (in_service / SLOTS_PRO_WOCHE) if SLOTS_PRO_WOCHE > 0 else 0.0
        queue_len_trace.append((env.now, q_len))
        util_trace.append((env.now, util))
        yield env.timeout(1.0)

# -------------------------
# 4) Run + Output
# -------------------------
def main():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    # Resource = Sitzungs-Slots pro Woche (Kapazität)
    slots = simpy.Resource(env, capacity=SLOTS_PRO_WOCHE)

    env.process(arrivals(env, slots, rng))
    env.process(monitor(env, slots))

    # Jahr + Puffer, damit Therapien fertig laufen
    env.run(until=WOCHEN_JAHR + THERAPIE_SITZUNGEN + 10)

    # Auswertung
    print("=== VT Simulation (alles-in-einem) ===")
    print(f"Slots pro Woche: {SLOTS_PRO_WOCHE:,}")
    print(f"Therapie: {THERAPIE_SITZUNGEN} Sitzungen (1/Woche)")
    print(f"Arrivals: Poisson(mean=BASE + growth*w), BASE={BASE_ARRIVALS_MEAN}, growth={GROWTH_PER_WEEK}/Woche")
    print(f"Start-Delay: {START_DELAY_MODE}")

    if not wait_times:
        print("\nKeine Patient*innen gestartet (oder Wartezeiten gemessen).")
        return

    wt = np.array(wait_times)
    print("\n--- Wartezeiten bis 1. Sitzung ---")
    print(f"Starts (gemessen): {len(wt):,}")
    print(f"Ø Wartezeit (Wochen): {wt.mean():.2f}")
    print(f"90% Quantil: {np.quantile(wt, 0.90):.2f}")
    print(f"95% Quantil: {np.quantile(wt, 0.95):.2f}")
    print(f"Max: {wt.max():.2f}")

    q = np.array([x[1] for x in queue_len_trace]) if queue_len_trace else np.array([])
    u = np.array([x[1] for x in util_trace]) if util_trace else np.array([])
    if len(q) and len(u):
        print("\n--- System-Trace (pro Woche) ---")
        print(f"Ø Queue-Länge: {q.mean():.1f}")
        print(f"Max Queue-Länge: {q.max():.0f}")
        print(f"Ø Auslastung: {u.mean()*100:.1f}%")
        print(f"Max Auslastung: {u.max()*100:.1f}%")

if __name__ == "__main__":
    main()

=== VT Simulation (alles-in-einem) ===
Slots pro Woche: 116,100
Therapie: 24 Sitzungen (1/Woche)
Arrivals: Poisson(mean=BASE + growth*w), BASE=2000, growth=30/Woche
Start-Delay: uniform

--- Wartezeiten bis 1. Sitzung ---
Starts (gemessen): 143,958
Ø Wartezeit (Wochen): 2.00
90% Quantil: 4.00
95% Quantil: 4.00
Max: 4.00

--- System-Trace (pro Woche) ---
Ø Queue-Länge: 0.0
Max Queue-Länge: 0
Ø Auslastung: 1.2%
Max Auslastung: 2.5%
